In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = 'user-top-read'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_top_tracks()

In [2]:
results.keys()
track_list = []
i = 0
for track in results['items']:
    track_list.append(track['external_urls']['spotify'])
    if i > 2:
        break
    i += 1

In [3]:
# s
# f = sp.audio_features(track_list)
# sp.search("Drake")


In [4]:
top_artists  = sp.current_user_top_artists()
artist_dict = {}
i = 0
for artist in top_artists['items']: 
    artist_dict[artist['name']] = [artist['images'][1]['url'], artist['followers']['total'], artist['genres'], artist['external_urls']['spotify'] ]
    if i > 2:
        break
    i += 1
# artist_dict


In [12]:
artist_top_tracks_dict = {}
for artist in artist_dict.values():
    top = sp.artist_top_tracks('artist[3]')['tracks']
    for track in top:
        print(track['id'])
        url = track['external_urls']['spotify']
        artist_top_tracks_dict[track['name']] = url
        break
    break
artist_top_tracks_dict

HTTP Error for GET to https://api.spotify.com/v1/artists/artist[3]/top-tracks returned 400 due to invalid id


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/artists/artist%5B3%5D/top-tracks?country=US:
 invalid id, reason: None

In [6]:
import pandas as pd
import numpy as np
import json
from datetime import date
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
artist_df = pd.read_csv('../../data/artists.csv')
tracks_df = pd.read_csv('../../data/tracks.csv')
with open("../../data/dict_artists.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [3]:
artist_df.drop(['id'], axis=1, inplace=True)

In [4]:
artist_df = artist_df[artist_df['genres'] != '[]']
artist_df = artist_df.sort_values(by=['popularity'], ascending=False)
l = len(artist_df)/15
artist_df = artist_df[:round(l)]
artist_df = artist_df.reset_index(drop=True)

In [5]:
artist_df

,followers,genres,name,popularity
0,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",Justin Bieber,100
1,54416812.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",Drake,98
2,32244734.0,"['latin', 'reggaeton', 'trap latino']",Bad Bunny,98
3,38869193.0,"['pop', 'post-teen pop']",Taylor Swift,98
4,16996777.0,"['chicago rap', 'melodic rap']",Juice WRLD,96
...,...,...,...,...
19903,2394.0,['russian edm'],Denis First,50
19904,1028316.0,"['sertanejo', 'sertanejo pop', 'sertanejo univ...",Jefferson Moraes,50
19905,33168.0,"['danish hip hop', 'danish pop']",Bro,50
19906,15823.0,"['bass house', 'deep groove house', 'electro h...",Marten Hørger,50


In [ ]:
# orig_model = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
# tfidf_matrix_orig = orig_model.fit_transform(artist_df['genres'])

In [6]:

model = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words=None)
tfidf_matrix = model.fit_transform(artist_df['genres'])

In [7]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 

In [8]:
indices = pd.Series(artist_df.index, index=artist_df['name'])

In [9]:
results = {}
for idx, row in artist_df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
    similar_items = [(cosine_similarities[idx][i], artist_df['name'][i]) for i in similar_indices] 
    results[row['name']] = similar_items[1:]

In [10]:
def _recommend(item_id, num):
    recs = results[item_id][:num]   
    preds = {}
    for pair in recs:
        preds[pair[1]] = pair[0]
    return preds

In [14]:
_recommend('Ed Sheeran', 5)

{'Lewis Capaldi': 1.0000000000000002,
 'Ed Sheeran': 1.0000000000000002,
 'Dua Lipa': 0.7212451083169278,
 'HONNE': 0.6876996711411917,
 'Louisa': 0.6867634867587893}

In [54]:
def _recommend_multiple(artists, num=10):
    dict_similar = {}
    for artist, weight in artists.items():
        dict_similar[artist] = _recommend(artist, num)
    artists_all = []
    for artist, similar_artists in dict_similar.items():
        artists_all.append(list(similar_artists.keys()))
    artists_unique = np.unique(artists_all).tolist()
    artists_dict = {artist: 0 for artist in artists_unique}
    for artist, similar_artists in dict_similar.items():
        for similar_artist, score in similar_artists.items():
            artists_dict[similar_artist] += artists[artist] * score
    return list({k: v for k, v in sorted(artists_dict.items(), key=lambda item: item[1], reverse=True) if k not in artists}.keys())[0:num]

In [ ]:
_recommend_multiple({"Drake": 10, "Queen": 8}, 5)

In [15]:
import pickle 
with open('cos_sim_results','wb') as f: pickle.dump(results, f)

In [4]:
with open('../pages/pickle/cos_sim_results', 'rb') as f:
    results = pickle.load(f)

dict